In [ ]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.5 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.1-cp310-cp310-linux_x86_64.whl size=662246 sha256=5282efbfe65371eebbce79473cc894bdc5ba8a1996729ae927de9789a64be0ae
  Stored in directory: /root/.cache/pip/wheels/d7/4a/51/d6dec3832025d601e2a365545e0972fbbeddbdb3eae72be96f
Successfully built pycuda


In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:
print("devices : ", cuda.Device.count())
print("device name : ", cuda.Device(0).name())

dev = cuda.Device(0)
print("compute capability : ", dev.compute_capability())
print("device name : ", dev.name())
print("device total memory : ", dev.total_memory() / 1024**3, "GB")



devices :  1
device name :  Tesla T4
compute capability :  (7, 5)
device name :  Tesla T4
device total memory :  14.74810791015625 GB


In [ ]:
'''
for memory allocation :
a_gpu = cuda.mem_alloc(a.nbytes)

for memory copy from host to devie :
cuda.memcpy_htod(a_gpu, a)

for memory copy from device to host :
cuda.memcpy_dtoh(a, a_gpu)

'''


'\nfor memory allocation :\na_gpu = cuda.mem_alloc(a.nbytes)\n\nfor memory copy from host to devie :\ncuda.memcpy_htod(a_gpu, a)\n\nfor memory copy from device to host :\ncuda.memcpy_dtoh(a, a_gpu)\n\n'

In [ ]:
import numpy as np
#for writing a kernel function
module = SourceModule("""
__global__ void multiply(float *dest , float *a , float *b )
{
    const int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if(idx < blockDim.x)
    {
        dest[idx] = a[idx] * b[idx];
    }
}
"""
)

multiply  = module.get_function("multiply")

a = np.random.rand(10).astype(np.float32)
b = np.random.rand(10).astype(np.float32)
c = np.zeros_like(a)

print(a)
print(b)

multiply(
    cuda.Out(c),
    cuda.In(a),
    cuda.In(b),
    block = (10,1,1),
    grid = (1,1)
)

print(c)




[0.54707897 0.8667677  0.75203145 0.4669701  0.88215697 0.99599504
 0.5162177  0.2081047  0.6370211  0.7753739 ]
[0.78329396 0.6947024  0.2883797  0.04133159 0.1759537  0.53936857
 0.47404242 0.8209888  0.24667253 0.687113  ]
[0.42852366 0.6021456  0.2168706  0.01930062 0.15521878 0.53720844
 0.24470909 0.17085162 0.1571356  0.53276944]


/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


In [ ]:
#calling the fucntion
'''
fx = module.get_function("add")
fx(a_gpu, b_gpu, c_gpu, block = (10,1,1), grid = (1,1))
'''


In [ ]:
#for more info about pycuda you can refer https://documen.tician.de/pycuda/